## Biblioteka Keras 

### Wstęp 

Keras jest biblioteką która bardzo ułatwia uczenie sieci neuronowych. Jej głównymi zaletami są proste, ale dające dużą kontrolę API oraz wysoka wydajność (która wynika z tego, że pod spodem `keras` korzysta z bardzo wydajnej biblioteki `tensorflow`.

In [3]:
import keras
import numpy as np
import pandas as pd
from keras import Sequential
from keras.layers import Dense,Activation

In [4]:
from sklearn.preprocessing import StandardScaler

Podstawowym obiektem z którego będziemy korzystać jest `Sequential`. Pozwala on na budowę podstawowych modeli, w których warstwy są ze sobą łączone sekwencyjnie.

Do budowania modelu potrzebujemy także warstw. Na razie będziemy korzystać z warstw `Dense` i `Activation`. Warstwy możecie zaimportować z modułu keras.layers

In [5]:
# wygenerujmy dane, na których będziemy na początku pracować
X = np.random.normal(size=(1000, 3))
y = np.sum(X * np.array([0.4, 2, -3]), axis=1, keepdims=True) -3 + np.random.normal(scale=0.5, size=(1000, 1))

Czas na zbudowanie modelu:

1. Najpierw inicjalizujemy pusty model:

In [4]:
model = Sequential()

2. Następnie dodajemy do niego warstwy. Zbudujmy na razie prosty model jednowarstwowy: 

In [5]:
#ponieważ jest to pierwsza warstwa modelu, musimy podać input_dim - czyli jaki jest rozmiar danych wejściowych
model.add(Dense(1,input_dim=3)) 

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


3. Kolejnym krokiem jest tzw. skompilowanie modelu. Tutaj podajemy informację o tym, jakiej funkcji straty chcemy użyć, jakie metryki mają być logowane podczas uczenia oraz w jaki sposób chcemy optymalizować wagi.

In [8]:
model.compile(loss='mean_squared_error',optimizer='sgd')

4. Teraz jesteśmy gotowi do nauczenia modelu. Służy do tego metoda `fit`:

In [12]:
model.fit(X,y,epochs=10,verbose=2)

Epoch 1/10
 - 0s - loss: 0.2443
Epoch 2/10
 - 0s - loss: 0.2444
Epoch 3/10
 - 0s - loss: 0.2442
Epoch 4/10
 - 0s - loss: 0.2442
Epoch 5/10
 - 0s - loss: 0.2443
Epoch 6/10
 - 0s - loss: 0.2444
Epoch 7/10
 - 0s - loss: 0.2443
Epoch 8/10
 - 0s - loss: 0.2445
Epoch 9/10
 - 0s - loss: 0.2443
Epoch 10/10
 - 0s - loss: 0.2443


In [13]:
y_pred = model.predict(X)

In [14]:
model.evaluate(X,y)

1000/1000 [==============================] - 0s 56us/step


0.24383300977945327

### Zbudujmy model na prawdziwych danych 

In [17]:
(x_train, y_train), (x_test, y_test) = keras.datasets.boston_housing.load_data()

In [18]:
scaler = StandardScaler()
x_train_st = scaler.fit_transform(x_train)
x_test_st = scaler.transform(x_test)

In [22]:
x_train.shape

(404, 13)

** Zadanie ** Zbuduj model z jedną warstwą ukrytą o 3 neuronach.

In [27]:
model_boston = Sequential()
model_boston.add(Dense(3,input_dim=x_train.shape[1]))
model_boston.add(Activation('relu'))
model_boston.add(Dense(1))
model_boston.compile(loss='mean_squared_error',optimizer='sgd')
model_boston.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 3)                 42        
_________________________________________________________________
activation_4 (Activation)    (None, 3)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 4         
Total params: 46
Trainable params: 46
Non-trainable params: 0
_________________________________________________________________


In [28]:
fit_history = model_boston.fit(x_train_st,y_train,batch_size=32,epochs=10,verbose=2)

Epoch 1/10
 - 0s - loss: 268.3702
Epoch 2/10
 - 0s - loss: 42.2363
Epoch 3/10
 - 0s - loss: 26.9770
Epoch 4/10
 - 0s - loss: 23.4837
Epoch 5/10
 - 0s - loss: 21.4442
Epoch 6/10
 - 0s - loss: 20.2106
Epoch 7/10
 - 0s - loss: 19.5641
Epoch 8/10
 - 0s - loss: 19.2219
Epoch 9/10
 - 0s - loss: 19.1973
Epoch 10/10
 - 0s - loss: 17.6998


In [29]:
fit_history.history

{'loss': [268.3702420149699,
  42.23632835161568,
  26.976994580561573,
  23.48368310456229,
  21.44419589845261,
  20.210601164562867,
  19.564101832928042,
  19.221944922267802,
  19.197265417268962,
  17.699761532320835]}

In [30]:
model_boston.evaluate(x_test_st,y_test)

102/102 [==============================] - 0s 729us/step


27.790628021838618

Dane są niestety dość małe, i praktycznie każde uruchomienie modelu daje nam inne rezultaty. Spróbujmy nauczyć model na większych danych.

In [33]:
train_set = pd.read_csv('Dane/adult/adult.data', sep=", ",header = None,engine='python')
test_set = pd.read_csv('Dane/adult/adult.test', sep=", ",skiprows = 1, header = None,engine='python') # Make sure to skip a row for the test set


col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

** Zadanie ** : przygotuj dane do uczenia - spokojnie możesz wyrzucić braki danych oraz kolumny o dużej ilości poziomów.

In [34]:
train_set.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
wage_class        0
dtype: int64

In [35]:
X_train = train_set.replace("?",np.nan).dropna()

In [36]:
X_test = test_set.replace("?",np.nan).dropna()

In [37]:
X_total = pd.concat([X_train,X_test])

In [38]:
X_total['wage_class'] = X_total.wage_class.replace({'<=50K':0,'<=50K.':0,'>50K':1,'>50K.':1})

In [11]:
# budowanie modelu

In [39]:
for col in X_total.select_dtypes(['object']).columns:
    print(col,len(X_total[col].unique()))

workclass 7
education 16
marital_status 7
occupation 14
relationship 6
race 5
sex 2
native_country 41


In [40]:
X_total.drop(['native_country','education','fnlwgt'],axis=1,inplace=True)

In [41]:
X_total = pd.get_dummies(X_total)
X_total.shape

(45222, 47)

In [42]:
X_train = X_total.iloc[:X_train.shape[0]]
X_test = X_total.iloc[X_train.shape[0]:]
y_train = X_train.pop('wage_class')
y_test = X_test.pop('wage_class')

In [43]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)# na szybko skalujemy wszystko
X_test_scaled = scaler.transform(X_test)

In [103]:
model = Sequential()
model.add(Dense(30,input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))#na wyjsciu dostajemy p-stwo
model.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])#to samo co loglos dla klasyfikacji
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 30)                1410      
_________________________________________________________________
activation_30 (Activation)   (None, 30)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 31        
_________________________________________________________________
activation_31 (Activation)   (None, 1)                 0         
Total params: 1,441
Trainable params: 1,441
Non-trainable params: 0
_________________________________________________________________


In [104]:
model.fit(X_train_scaled,y_train,epochs=10,verbose=2)

Epoch 1/10
 - 2s - loss: 0.4135 - acc: 0.8050
Epoch 2/10
 - 2s - loss: 0.3524 - acc: 0.8370
Epoch 3/10
 - 2s - loss: 0.3421 - acc: 0.8402
Epoch 4/10
 - 2s - loss: 0.3366 - acc: 0.8426
Epoch 5/10
 - 2s - loss: 0.3325 - acc: 0.8442
Epoch 6/10
 - 2s - loss: 0.3293 - acc: 0.8445
Epoch 7/10
 - 1s - loss: 0.3267 - acc: 0.8463
Epoch 8/10
 - 2s - loss: 0.3246 - acc: 0.8477
Epoch 9/10
 - 2s - loss: 0.3227 - acc: 0.8487
Epoch 10/10
 - 2s - loss: 0.3213 - acc: 0.8493


In [105]:
model.evaluate(X_test_scaled, y_test)

15060/15060 [==============================] - 1s 65us/step


[0.32915633133999694, 0.8482735723613268]

** Zadanie **: zbuduj w kerasie następujące sieci (i zbadaj ich performance na danych testowych):

* model z jedną warstwą ukrytą o 40 neuronach i aktywacją 'tanh'


In [84]:
X_train.shape

(30162, 46)

In [97]:
model = Sequential()
model.add(Dense(40,input_dim = X_train.shape[1]))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 40)                1880      
_________________________________________________________________
activation_26 (Activation)   (None, 40)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 41        
_________________________________________________________________
activation_27 (Activation)   (None, 1)                 0         
Total params: 1,921
Trainable params: 1,921
Non-trainable params: 0
_________________________________________________________________


In [98]:
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [99]:
model.fit(X_train_scaled,y_train,epochs=10,verbose=2)

Epoch 1/10
 - 2s - loss: 0.4244 - acc: 0.7965
Epoch 2/10
 - 2s - loss: 0.3442 - acc: 0.8411
Epoch 3/10
 - 2s - loss: 0.3357 - acc: 0.8434
Epoch 4/10
 - 2s - loss: 0.3320 - acc: 0.8449
Epoch 5/10
 - 1s - loss: 0.3296 - acc: 0.8466
Epoch 6/10
 - 2s - loss: 0.3277 - acc: 0.8469
Epoch 7/10
 - 2s - loss: 0.3262 - acc: 0.8481
Epoch 8/10
 - 1s - loss: 0.3250 - acc: 0.8479
Epoch 9/10
 - 1s - loss: 0.3240 - acc: 0.8478
Epoch 10/10
 - 1s - loss: 0.3231 - acc: 0.8487


In [100]:
model.evaluate(X_test_scaled,y_test)

15060/15060 [==============================] - 1s 63us/step


[0.3281439783999011, 0.8460159362391488]

* model z dwoma warstwami ukrytymi o odpowiednio 40 i 20 neuronach (aktywacja 'relu')

In [106]:
model2 = Sequential()
model2.add(Dense(40,input_dim = X_train_scaled.shape[1]))
model2.add(Activation('relu'))
model2.add(Dense(20))
model2.add(Activation('relu'))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))
model2.compile(optimizer='sgd',loss='binary_crossentropy',metrics=["accuracy"])

In [107]:
model2.fit(X_train_scaled,y_train,epochs=10,verbose=2)

Epoch 1/10
 - 2s - loss: 0.4364 - acc: 0.7971
Epoch 2/10
 - 2s - loss: 0.3508 - acc: 0.8369
Epoch 3/10
 - 2s - loss: 0.3397 - acc: 0.8417
Epoch 4/10
 - 2s - loss: 0.3338 - acc: 0.8441
Epoch 5/10
 - 2s - loss: 0.3292 - acc: 0.8464
Epoch 6/10
 - 2s - loss: 0.3259 - acc: 0.8473
Epoch 7/10
 - 2s - loss: 0.3230 - acc: 0.8497
Epoch 8/10
 - 2s - loss: 0.3206 - acc: 0.8506
Epoch 9/10
 - 2s - loss: 0.3187 - acc: 0.8509
Epoch 10/10
 - 2s - loss: 0.3170 - acc: 0.8523


In [108]:
model2.evaluate(X_test_scaled,y_test)

15060/15060 [==============================] - 1s 72us/step


[0.3278095884270877, 0.8479415670492418]

### Co możemy pozmieniać w naszym modelu?

Odpowiedź - dosłownie wszystko. Ale zaczniemy najpierw od tego, co poznaliśmy wcześniej.

#### Learning rate 

Podając `'sgd'` do metody `compile` używamy domyślnych parametrów danego algorytmu. Jeśli chcemy je zmodyfikować, musimy zaimportować odpowiednią klasę.

In [67]:
from keras.optimizers import SGD

In [68]:
??SGD

In [109]:
custom_sgd = SGD(lr=0.1)

In [110]:
model = Sequential()
model.add(Dense(30,input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))#na wyjsciu dostajemy p-stwo
model.compile(loss='binary_crossentropy',optimizer= custom_sgd,metrics=["accuracy"])#to samo co loglos dla klasyfikacji

In [111]:
model.fit(X_train_scaled,y_train,epochs=10,verbose=2)

Epoch 1/10
 - 2s - loss: 0.3519 - acc: 0.8352
Epoch 2/10
 - 2s - loss: 0.3251 - acc: 0.8478
Epoch 3/10
 - 2s - loss: 0.3195 - acc: 0.8498
Epoch 4/10
 - 2s - loss: 0.3163 - acc: 0.8523
Epoch 5/10
 - 2s - loss: 0.3143 - acc: 0.8532
Epoch 6/10
 - 2s - loss: 0.3136 - acc: 0.8540
Epoch 7/10
 - 2s - loss: 0.3125 - acc: 0.8549
Epoch 8/10
 - 2s - loss: 0.3115 - acc: 0.8547
Epoch 9/10
 - 2s - loss: 0.3111 - acc: 0.8554
Epoch 10/10
 - 2s - loss: 0.3099 - acc: 0.8549


In [112]:
model.evaluate(X_test_scaled,y_test)

15060/15060 [==============================] - 1s 70us/step


[0.32418408065361487, 0.8488711819230798]

#### Early stopping 

W kerasie mamy dostępne tzw. *callbacki* - są to operacje, które będą wywoływane w odpowiednich momentach uczenia (na przykład na koniec batcha, albo na koniec epoki). Jednym z takich callbacków jest właśnie EarlyStopping - na koniec batcha liczy on metryki na zbiorze walidacyjnym, i w przypadku pogorszenia się wyników kończy uczenie.

In [73]:
from keras.callbacks import EarlyStopping

In [74]:
EarlyStopping??

In [113]:
model = Sequential()
model.add(Dense(30,input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))#na wyjsciu dostajemy p-stwo
model.compile(loss='binary_crossentropy',optimizer= custom_sgd,metrics=["accuracy"])#to samo co loglos dla klasyfikacji

callbacks = [EarlyStopping(min_delta=0.005)]
model.fit(X_train_scaled,y_train,epochs=10,verbose=2,callbacks=callbacks,validation_split=0.3)

Train on 21113 samples, validate on 9049 samples
Epoch 1/10
 - 2s - loss: 0.3569 - acc: 0.8315 - val_loss: 0.3361 - val_acc: 0.8452
Epoch 2/10
 - 2s - loss: 0.3289 - acc: 0.8461 - val_loss: 0.3245 - val_acc: 0.8489
Epoch 3/10
 - 2s - loss: 0.3218 - acc: 0.8493 - val_loss: 0.3257 - val_acc: 0.8463


In [114]:
model.evaluate(X_test_scaled,y_test)

15060/15060 [==============================] - 1s 41us/step


[0.33489499862687994, 0.8445551128659748]

#### Zapisywanie modelu

Przy okazji callbacków - mamy dostępny callback, który zapisuje nasz model na dysku.

In [77]:
from keras.callbacks import ModelCheckpoint

In [78]:
ModelCheckpoint??

In [79]:
model = Sequential()
model.add(Dense(30,input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))#na wyjsciu dostajemy p-stwo
model.compile(loss='binary_crossentropy',optimizer= custom_sgd)#to samo co loglos dla klasyfikacji

callbacks = [EarlyStopping(min_delta=0.005),
             ModelCheckpoint(filepath='./my_nn_model.h5',save_best_only=True)]

model.fit(X_train_scaled,y_train,epochs=15,verbose=2,callbacks=callbacks,validation_split=0.3)

Train on 21113 samples, validate on 9049 samples
Epoch 1/15
 - 2s - loss: 0.3564 - val_loss: 0.3337
Epoch 2/15
 - 1s - loss: 0.3282 - val_loss: 0.3253
Epoch 3/15
 - 1s - loss: 0.3214 - val_loss: 0.3225


In [80]:
model.load_weights('./my_nn_model.h5')

In [81]:
from keras.models import load_model

In [82]:
loaded_model = load_model('./my_nn_model.h5')# nie trzeba kompilować

In [83]:
loaded_model.predict(X_test_scaled)

array([[4.0252708e-04],
       [9.0350673e-02],
       [5.6612581e-01],
       ...,
       [6.8327868e-01],
       [1.0991303e-01],
       [8.0861509e-01]], dtype=float32)